In [ ]:
# src/metrics.py

import numpy as np
from libpysal.weights import Queen
from esda.moran import Moran

def gini_coefficient(x):
    """Compute Gini coefficient for a 1D array."""
    x = np.array(x, dtype=float)
    x = x[np.isfinite(x)]
    x = x[x >= 0]
    if len(x) == 0:
        return np.nan
    x_sorted = np.sort(x)
    n = len(x_sorted)
    S = x_sorted.sum()
    cumx = np.cumsum(x_sorted)
    return 1 + 1.0/n - 2.0 * np.sum(cumx) / (n * S)

def calc_gini_moran_clean(gdf, col_name):
    """
    Calculate Gini and Global Moran's I for one column in a GeoDataFrame.
    """
    s = gdf[col_name].astype(float).replace([np.inf, -np.inf], np.nan).dropna()
    g_sub = gdf.loc[s.index].copy()

    w = Queen.from_dataframe(g_sub, use_index=False)
    w.transform = "r"

    y = s.values

    gini = gini_coefficient(y)
    moran = Moran(y, w)

    z_val = float(np.array(moran.z).flatten()[0])
    p_val = float(np.array(moran.p_norm).flatten()[0])

    return {
        "gini": float(gini),
        "moran_I": float(moran.I),
        "z_score": z_val,
        "p_value": p_val
    }
